In [26]:
import pandas as pd
import math
import numpy as np
import os
from sklearn import preprocessing
# print(os.getcwd())

In [27]:
# CHANGE the file name of which csv to modify as variable data
# CHANGE working directory as variable wd
# CHANGE variable sample_ratio

data = pd.read_csv('train.csv')

# # baseline data
# data = pd.read_csv(wd + 'train.csv') # train.csv is from Kaggle
# data['CityId'] = ''

# remove Philadelphia
data.loc[data.City=='Philadelphia','CityId'] = 4
data.loc[data.City=='Atlanta','CityId'] = 3
data.loc[data.City=='Boston','CityId'] = 2
data.loc[data.City=='Chicago','CityId'] = 1

In [28]:
# print(data.shape)
# data.head()
# There are only 235 entries from January and 5 entries from May
data.groupby(['Month']).count()
# dropping January and May entries (don't drop)
# data = data[data.Month != 1]
# data = data[data.Month != 5]

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,CityId
Month,,,,,,,,,,,,,,,,,,,,,
1,869,869,869,869,866,866,869,869,869,869,...,869,869,869,869,869,869,869,869,869,869
5,164,164,164,164,164,164,164,164,164,164,...,164,164,164,164,164,164,164,164,164,164
6,110188,110188,110188,110188,109173,109473,110188,110188,110188,110188,...,110188,110188,110188,110188,110188,110188,110188,110188,110188,110188
7,110810,110810,110810,110810,109791,110154,110810,110810,110810,110810,...,110810,110810,110810,110810,110810,110810,110810,110810,110810,110810
8,122784,122784,122784,122784,121613,121982,122784,122784,122784,122784,...,122784,122784,122784,122784,122784,122784,122784,122784,122784,122784
9,124782,124782,124782,124782,123629,123979,124782,124782,124782,124782,...,124782,124782,124782,124782,124782,124782,124782,124782,124782,124782
10,133039,133039,133039,133039,131704,132135,133039,133039,133039,133039,...,133039,133039,133039,133039,133039,133039,133039,133039,133039,133039
11,125227,125227,125227,125227,124003,124410,125227,125227,125227,125227,...,125227,125227,125227,125227,125227,125227,125227,125227,125227,125227
12,129546,129546,129546,129546,128277,128712,129546,129546,129546,129546,...,129546,129546,129546,129546,129546,129546,129546,129546,129546,129546


In [29]:
# sample unique intersections proportionally

sample = pd.DataFrame(columns = data.columns)
# ratio/percentage of samples to get/work with
# e.g. float(10000)/float(468852) = 0.021
sample_ratio = 0.05
np.random.seed(1234)

for i in [1,2,3,4]:
    IntersectionIds = data.loc[data.CityId==i,'IntersectionId'].unique()
    print(len(IntersectionIds))
    index = np.random.choice(IntersectionIds, size = int(len(IntersectionIds)*sample_ratio), replace=False)
    # in that city and in that intersection
    s = data[(data.CityId==i) & (data.IntersectionId.isin(index))]
    sample = sample.append(s)
    print(sample.shape)

# sample.iloc[:20, :]

2129
(7119, 29)
973
(15745, 29)
377
(20839, 29)
1317
(38818, 29)


### Encoding

In [30]:
print(sample.shape)
new_sample = sample.copy()

(38818, 29)


In [31]:
# drop the intersection ids whose counts/occurrence are 1
IntersectionIds = data.loc[:,'IntersectionId'].unique()
intersection_id_counts = sample["IntersectionId"].value_counts().to_dict()
drop_ids = [inter_id for inter_id, count in intersection_id_counts.items() if count == 1]
drop_ix = []
for i in drop_ids:
    ix = sample.index[sample['IntersectionId'] == i].tolist()
    drop_ix.append(ix[0]) # because there's only 1 occurrence
new_sample = sample.drop(drop_ix)
print(new_sample.shape)

(38817, 29)


In [32]:
# add distance from city centers
# def add_distance(df): 
#     df_center = pd.DataFrame({"Atlanta":[33.753746, -84.386330],
#                              "Boston":[42.361145, -71.057083],
#                              "Chicago":[41.881832, -87.623177],
#                              "Philadelphia":[39.9520, -75.1640]})
    
#     df["CenterDistance"] = df.apply(lambda row: math.sqrt((df_center[row.City][0] - row.Latitude) ** 2 +
#                                                           (df_center[row.City][1] - row.Longitude) ** 2) , axis=1)

# add_distance(new_sample)

In [33]:
# train, test split data
from sklearn.model_selection import train_test_split
train_sample, test_sample = train_test_split(new_sample, test_size=0.2, stratify = new_sample['IntersectionId'])

In [34]:
train_sample

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,CityId
573609,2493944,506,39.95001,-75.15706,Chestnut Street,Chestnut Street,E,E,20,1,...,0,0,15,0.0,0.0,0.0,0.0,58.8,Philadelphia,4.0
147572,2067907,498,33.74432,-84.40842,Northside Drive Southwest,Chapel Street Southwest,SW,SW,22,1,...,0,0,0,0.0,0.0,0.0,0.0,0.0,Atlanta,3.0
482887,2403222,54,39.97039,-75.14654,West Girard Avenue,North 6th Street,W,S,15,1,...,0,0,21,0.0,0.0,0.0,0.0,62.4,Philadelphia,4.0
491437,2411772,96,39.95258,-75.14848,Arch Street,North Independence Ml East,W,N,6,0,...,24,29,40,33.9,43.7,50.0,56.0,58.5,Philadelphia,4.0
491455,2411790,96,39.95258,-75.14848,Arch Street,Arch Street,W,W,17,0,...,0,0,30,0.0,0.0,0.0,0.0,67.7,Philadelphia,4.0
34415,1954750,124,33.77360,-84.36371,Ponce de Leon Avenue Northeast,Ponce de Leon Avenue Northeast,W,W,12,1,...,0,0,39,0.0,0.0,0.0,0.0,95.3,Atlanta,3.0
812367,2732702,1765,39.95550,-75.21365,South 46th Street,South 46th Street,N,N,8,0,...,49,50,67,61.4,64.9,73.4,90.7,97.5,Philadelphia,4.0
104871,2025206,345,33.71820,-84.42520,Lee Street Southwest,Campbellton Road Southwest,S,W,23,0,...,0,0,70,0.0,0.0,0.0,0.0,489.1,Atlanta,3.0
344733,2265068,219,41.89869,-87.74590,North Cicero Avenue,North Cicero Avenue,S,S,12,1,...,0,19,25,0.0,0.0,0.0,77.1,96.2,Chicago,1.0
286721,2207056,877,42.34951,-71.07950,Boylston Street,Boylston Street,E,E,18,1,...,27,31,35,49.8,64.6,70.4,70.5,78.5,Boston,2.0


In [35]:
#replace StreetName of NaN with Unknown
pd.DataFrame.dropna(train_sample,inplace=True)
pd.DataFrame.dropna(test_sample,inplace=True)

/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
street_type = ["Street", "St", "Avenue", "Ave", "Boulevard", "Road", "Drive", "Lane",
              "Tunnel", "Highway", "Way", "Parkway", "Parking", "Oval", "Square",
              "Place", "Bridge", "Unknown"]

street_type_num = [0, 0, 1, 1, 2, 3, 4, 5,
                     6, 7, 8, 9, 9, 10, 11,
                     12, 13, 14]
def get_street_type(row, column):
    for i, s in enumerate(street_type):
        if s in row[column]:
            return street_type_num[i]
    return 15

def extract_street_type(df, column):
    df[column.replace("StreetName", "Type")] = df.apply(lambda row: get_street_type(row, column), axis=1)



In [37]:
extract_street_type(train_sample, "EntryStreetName")

/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
extract_street_type(train_sample, "ExitStreetName")
extract_street_type(test_sample, "EntryStreetName")
extract_street_type(test_sample, "ExitStreetName")

/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [39]:
directions={"E":0, "SE":1, "S":2, "SW":3, "W":4, "NW":5, "N":6, "NE":7,
           0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7}

train_sample["EntryHeading"] = train_sample["EntryHeading"].map(directions).astype(np.int32)
train_sample["ExitHeading"] = train_sample["ExitHeading"].map(directions).astype(np.int32)

test_sample["EntryHeading"] = test_sample["EntryHeading"].map(directions).astype(np.int32)
test_sample["ExitHeading"] = test_sample["ExitHeading"].map(directions).astype(np.int32)

def calc_turn(dataframe):
    dataframe["RightTurn"] = ((dataframe["EntryHeading"] - dataframe["ExitHeading"]) % 8 == 2).astype(np.int32)
    dataframe["LeftTurn"] = ((dataframe["EntryHeading"] - dataframe["ExitHeading"]) % 8 == 6).astype(np.int32)
    dataframe["PassThru"] = ((dataframe["EntryHeading"] - dataframe["ExitHeading"]) % 8 == 4).astype(np.int32)
    dataframe["UTurn"] = (dataframe["EntryHeading"] - dataframe["ExitHeading"] == 0).astype(np.int32)
    
    dataframe["RightSide"] = ((dataframe["ExitHeading"] - dataframe["EntryHeading"]) % 8 > 5).astype(np.int32)
    dataframe["LeftSide"] = ((dataframe["ExitHeading"] - (dataframe["EntryHeading"] + 1)) % 8 < 3).astype(np.int32)
    dataframe["Direction"] = ((dataframe["ExitHeading"] - (dataframe["EntryHeading"])) % 8).astype(np.int32)

calc_turn(train_sample)
calc_turn(test_sample)

/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [40]:
train_sample.head()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,CityId,EntryType,ExitType,RightTurn,LeftTurn,PassThru,UTurn,RightSide,LeftSide,Direction
573609,2493944,506,39.95001,-75.15706,Chestnut Street,Chestnut Street,0,0,20,1,...,4.0,0,0,0,0,0,1,0,0,0
147572,2067907,498,33.74432,-84.40842,Northside Drive Southwest,Chapel Street Southwest,3,3,22,1,...,3.0,4,0,0,0,0,1,0,0,0
482887,2403222,54,39.97039,-75.14654,West Girard Avenue,North 6th Street,4,2,15,1,...,4.0,1,0,1,0,0,0,1,0,6
491437,2411772,96,39.95258,-75.14848,Arch Street,North Independence Ml East,4,6,6,0,...,4.0,0,15,0,1,0,0,0,1,2
491455,2411790,96,39.95258,-75.14848,Arch Street,Arch Street,4,4,17,0,...,4.0,0,0,0,0,0,1,0,0,0


In [41]:
# dropping some features and splitting train/test features and labels

X_train = train_sample.drop(['RowId','EntryStreetName','Path',
       'ExitStreetName','TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80'],axis=1)

labels = ['TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80']
y_train = train_sample[labels]
# X_train.rename(columns={'Cityid':'CityId'}, inplace=True);

In [42]:
test_sample = test_sample.dropna()
X_test = test_sample.drop(['RowId','EntryStreetName','Path',
       'ExitStreetName','TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80'],axis=1)
y_test = test_sample[labels]
# X_test.rename(columns={'Cityid':'CityId'}, inplace=True);

In [43]:
# OHE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# drop City column (don't rerun)
X_train = X_train.drop(['City'], axis=1)
X_test = X_test.drop(['City'], axis=1)

In [44]:
# which ones are categorical features
categorical_cols = ['CityId','EntryHeading', 'ExitHeading']
categorical_feature_mask = [True if c in categorical_cols else False for c in X_train.columns]
categorical_feature_mask = pd.Series(data=categorical_feature_mask, index=X_train.columns)

# label encoding
encoder_train = LabelEncoder()
X_train[categorical_cols] = X_train[categorical_cols].apply(encoder_train.fit_transform)
encoder_test = LabelEncoder()
X_test[categorical_cols] = X_test[categorical_cols].apply(encoder_test.fit_transform)


# instantiate OneHotEncoder on categ cols only
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix
# ohe = OneHotEncoder(categorical_features = categorical_feature_mask, sparse=False) 
ohe = OneHotEncoder(sparse=False) 
# apply OneHotEncoder on categorical feature columns
X_ohe_train_ary = ohe.fit_transform(X_train[categorical_cols]) # returns a numpy array
X_ohe_test_ary = ohe.fit_transform(X_test[categorical_cols])

/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/huajunzhou/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categori

In [45]:
X_ohe_test_ary

array([[0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [46]:
# expect 19 additional columns to original data because of OHE, while dropping categorical columns
print(X_train["CityId"].unique()) # 3 unique
print(X_train["EntryHeading"].unique()) # 8 unique
print(X_train["ExitHeading"].unique()) # 8 unique
# shape of dataframes and np array
print(X_train.shape)
print(X_test.shape)
# print(X_train[categorical_cols].shape)
# print(X_test[categorical_cols].shape)
print(X_ohe_train_ary.shape)
print(X_ohe_test_ary.shape)

[3 2 0 1]
[0 3 4 6 2 1 7 5]
[0 3 2 6 4 1 7 5]
(30538, 18)
(7623, 18)
(30538, 20)
(7623, 20)


In [47]:
# append OHE vectors to dataframes
num_unique_CityId = len(X_train["CityId"].unique())
num_unique_EntryHeading = len(X_train["EntryHeading"].unique())
num_unique_ExitHeading = len(X_train["ExitHeading"].unique())

for i in range(num_unique_CityId):
    # add new column where the column has 0/1 values for that feature
    X_train[categorical_cols[0] + str(i)] = X_ohe_train_ary[:, i]
    X_test[categorical_cols[0] + str(i)] = X_ohe_test_ary[:, i]
for i in range(num_unique_EntryHeading):
    X_train[categorical_cols[1] + str(i)] = X_ohe_train_ary[:, i+num_unique_CityId]
    X_test[categorical_cols[1] + str(i)] = X_ohe_test_ary[:, i+num_unique_CityId]
for i in range(num_unique_ExitHeading):
    X_train[categorical_cols[2] + str(i)] = X_ohe_train_ary[:, i+num_unique_CityId+num_unique_EntryHeading]
    X_test[categorical_cols[2] + str(i)] = X_ohe_test_ary[:, i+num_unique_CityId+num_unique_EntryHeading]

In [48]:
# sort column names, drop categorical cols, and save to csv
categorical_cols = ['CityId','EntryHeading', 'ExitHeading', 'IntersectionId']
X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_train = X_train.drop(categorical_cols, axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1)
X_test = X_test.drop(categorical_cols, axis=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train.head(10)

(30538, 34)
(7623, 34)
(30538, 15)
(7623, 15)


,CityId0,CityId1,CityId2,CityId3,Direction,EntryHeading0,EntryHeading1,EntryHeading2,EntryHeading3,EntryHeading4,...,Latitude,LeftSide,LeftTurn,Longitude,Month,PassThru,RightSide,RightTurn,UTurn,Weekend
573609,0.0,0.0,0.0,1.0,0,1.0,0.0,0.0,0.0,0.0,...,39.95001,0,0,-75.15706,10,0,0,0,1,1
147572,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,1.0,0.0,...,33.74432,0,0,-84.40842,6,0,0,0,1,1
482887,0.0,0.0,0.0,1.0,6,0.0,0.0,0.0,0.0,1.0,...,39.97039,0,0,-75.14654,9,0,1,1,0,1
491437,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,...,39.95258,1,1,-75.14848,6,0,0,0,0,0
491455,0.0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,1.0,...,39.95258,0,0,-75.14848,6,0,0,0,1,0
34415,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,1.0,...,33.77360,0,0,-84.36371,10,0,0,0,1,1
812367,0.0,0.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,...,39.95550,0,0,-75.21365,10,0,0,0,1,0
104871,0.0,0.0,1.0,0.0,2,0.0,0.0,1.0,0.0,0.0,...,33.71820,1,1,-84.42520,8,0,0,0,0,0
344733,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,...,41.89869,0,0,-87.74590,9,0,0,0,1,1
286721,0.0,1.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,...,42.34951,0,0,-71.07950,10,0,0,0,1,1


In [49]:
# saving
X_train.to_csv("formatted_train_data_sampleratio1{}.csv".format(sample_ratio), encoding='utf-8', index=False)
X_test.to_csv("formatted_test_data_sampleratio1{}.csv".format(sample_ratio), encoding='utf-8', index=False)
y_train.to_csv("formatted_train_labels_sampleratio1{}.csv".format(sample_ratio), encoding='utf-8', index=False)
y_test.to_csv("formatted_test_labels_sampleratio1{}.csv".format(sample_ratio), encoding='utf-8', index=False)

# X_train.to_csv(wd + "baseline_train_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# X_test.to_csv(wd + "baseline_test_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# y_train.to_csv(wd + "baseline_train_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# y_test.to_csv(wd + "baseline_test_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)